<a href="https://colab.research.google.com/github/average81/KION/blob/scene_segmentation/Subs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

субтитры группируются в "сцены" по двум критериям: паузе во времени между субтитрами и семантической (смысловой) схожести текста с помощью spaCy



    Загружается модель spaCy для русского языка (ru_core_news_md), чтобы анализировать смысл текста субтитров.

    Считывается и парсится файл субтитров SRT.

    Каждая реплика субтитров превращается в объект с таймкодами, текстом и лингвистическим представлением.

    Реплики группируются в сцены на основе временных разрывов между репликами и семантической схожести текста.

    Проводится дополнительное объединение сцен, чтобы избежать слишком коротких сцен или искусственного дробления.

    Итоговые сцены выводятся на экран и сохраняются в текстовый файл.


In [1]:
pip install pysrt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 694.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysrt: filename=pysrt-1.1.2-py3-none-any.whl size=13443 sha256=aac3d3d13eea2a8a9bb3dd14342cf9e7be970246c71c6255e5ed86c5876f109d
  Stored in directory: /root/.cache/pip/wheels/2d/b2/df/ea10959920533975b4a74a25a35e6d79655b63f3006611a99f
Successfully built pysrt


In [1]:
!python -m spacy download ru_core_news_md

  Using cached https://github.com/explosion/spacy-models/releases/download/ru_core_news_md-3.8.0/ru_core_news_md-3.8.0-py3-none-any.whl (41.9 MB)
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
import spacy
from dataclasses import dataclass
from typing import List, Tuple
import re
from statistics import mean

# Настройки
MIN_SCENE_DURATION = 10.0  # Минимальная длительность сцены в секундах
TIME_GAP_THRESHOLD = 3.0  # Максимальный допустимый разрыв между репликами
SIMILARITY_THRESHOLD = 0.55  # Порог семантической схожести
MAX_SCENE_DURATION = 240.0  # Максимальная длительность одной сцены

# Загружаем модель для русского языка
try:
    nlp = spacy.load("ru_core_news_md")
except OSError:
    print("Модель ru_core_news_md не найдена. Установите её командой:")
    print("python -m spacy download ru_core_news_md")
    exit()

@dataclass
class SubtitleLine:
    index: int
    start: float  # в секундах
    end: float    # в секундах
    text: str
    doc: any = None  # spaCy Doc объект

def read_srt_file(file_path: str) -> str:
    """Читает содержимое файла SRT"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Файл {file_path} не найден!")
        exit()
    except UnicodeDecodeError:
        try:
            with open(file_path, 'r', encoding='cp1251') as file:
                return file.read()
        except Exception as e:
            print(f"Ошибка при чтении файла: {e}")
            exit()

def parse_srt(srt_text: str) -> List[SubtitleLine]:
    subtitles = []
    blocks = re.split(r'\n\s*\n', srt_text.strip())

    for block in blocks:
        lines = block.strip().split('\n')
        if len(lines) < 3:
            continue

        try:
            index = int(lines[0])
            time_match = re.match(r'(\d{2}):(\d{2}):(\d{2}),(\d{3}) --> (\d{2}):(\d{2}):(\d{2}),(\d{3})', lines[1])
            if not time_match:
                continue

            h1, m1, s1, ms1 = map(int, time_match.groups()[:4])
            h2, m2, s2, ms2 = map(int, time_match.groups()[4:8])
            start = h1 * 3600 + m1 * 60 + s1 + ms1 / 1000
            end = h2 * 3600 + m2 * 60 + s2 + ms2 / 1000

            text = '\n'.join(lines[2:])
            doc = nlp(text)  # Анализируем текст с помощью spaCy
            subtitles.append(SubtitleLine(index, start, end, text, doc))
        except Exception as e:
            print(f"Ошибка при обработке блока: {e}")
            continue

    return subtitles

def calculate_scene_metrics(scene: List[SubtitleLine]) -> Tuple[float, float]:
    """Вычисляет длительность сцены и среднюю схожесть реплик"""
    if not scene:
        return 0.0, 0.0

    durations = [sub.end - sub.start for sub in scene]
    similarities = []

    for i in range(1, len(scene)):
        sim = scene[i-1].doc.similarity(scene[i].doc)
        similarities.append(sim)

    total_duration = scene[-1].end - scene[0].start
    avg_similarity = mean(similarities) if similarities else 1.0

    return total_duration, avg_similarity

def should_merge_scenes(prev_scene: List[SubtitleLine], current_scene: List[SubtitleLine]) -> bool:
    """Определяет, нужно ли объединять сцены"""
    if not prev_scene or not current_scene:
        return False

    # Проверяем временной промежуток между сценами
    time_gap = current_scene[0].start - prev_scene[-1].end

    # Проверяем схожесть последней реплики предыдущей сцены и первой текущей
    similarity = prev_scene[-1].doc.similarity(current_scene[0].doc)

    # Вычисляем общую длительность объединенной сцены
    merged_duration = current_scene[-1].end - prev_scene[0].start

    # Объединяем, если:
    # 1. Небольшой временной разрыв И хорошая схожесть
    # 2. ИЛИ если одна из сцен слишком короткая
    # 3. И при этом объединенная сцена не станет слишком длинной
    return ((time_gap <= TIME_GAP_THRESHOLD and similarity >= SIMILARITY_THRESHOLD) or
            any(calculate_scene_metrics(s)[0] < MIN_SCENE_DURATION for s in [prev_scene, current_scene])) and \
           merged_duration <= MAX_SCENE_DURATION

def group_into_scenes(subtitles: List[SubtitleLine]) -> List[List[SubtitleLine]]:
    if not subtitles:
        return []

    # Сначала группируем по простым правилам
    initial_scenes = []
    current_scene = [subtitles[0]]

    for prev_sub, curr_sub in zip(subtitles, subtitles[1:]):
        time_gap = curr_sub.start - prev_sub.end
        similarity = prev_sub.doc.similarity(curr_sub.doc) if prev_sub.doc and curr_sub.doc else 0

        if time_gap > TIME_GAP_THRESHOLD or similarity < SIMILARITY_THRESHOLD:
            initial_scenes.append(current_scene)
            current_scene = [curr_sub]
        else:
            current_scene.append(curr_sub)

    initial_scenes.append(current_scene)

    # Затем объединяем короткие или связанные сцены
    merged_scenes = []

    for scene in initial_scenes:
        if not merged_scenes:
            merged_scenes.append(scene)
            continue

        if should_merge_scenes(merged_scenes[-1], scene):
            merged_scenes[-1].extend(scene)
        else:
            merged_scenes.append(scene)

    return merged_scenes

def print_scenes(scenes: List[List[SubtitleLine]]):
    for i, scene in enumerate(scenes, 1):
        duration = scene[-1].end - scene[0].start
        num_dialogs = len(scene)
        print(f"\n=== Сцена {i} ({duration:.1f} сек, {num_dialogs} реплик) ===")
        for sub in scene:
            print(f"{sub.start:.1f}-{sub.end:.1f}: {sub.text}")

# Основной код
if __name__ == "__main__":
    # Укажите путь к вашему файлу субтитров
    srt_file_path = "Mr_And_Mrs_Smith_RUS_2005.srt"

    # Читаем файл
    srt_text = read_srt_file(srt_file_path)

    # Парсим и анализируем
    subtitles = parse_srt(srt_text)
    print(f"Обработано {len(subtitles)} реплик")

    scenes = group_into_scenes(subtitles)
    print(f"Выделено {len(scenes)} сцен")

    # Выводим результат
    print_scenes(scenes)

    # Сохраняем результат в файл
    with open("scenes_output.txt", "w", encoding="utf-8") as f:
        for i, scene in enumerate(scenes, 1):
            duration = scene[-1].end - scene[0].start
            f.write(f"\n=== Сцена {i} ({duration:.1f} сек) ===\n")
            for sub in scene:
                f.write(f"{sub.start:.2f}-{sub.end:.2f}: {sub.text}\n")
    print("\nРезультат также сохранён в файл scenes_output.txt")

Обработано 1079 реплик
Выделено 53 сцен

=== Сцена 1 (184.2 сек, 47 реплик) ===
26.6-29.2: - Давайте, я первый.
- Да.
29.4-31.7: Я думаю, что зря мы сюда пришли.
33.8-35.8: - Мы женаты уже пять лет.
- Шесть.
36.3-38.0: Пять или шесть лет.
38.8-41.7: И для нас это,
как ТО для машины.
41.9-44.8: Диагностика, проверка движка,
45.1-48.6: поменять масло,
пару сальников.
51.9-54.4: Что ж, давайте откроем капот.
55.8-58.3: На сколько вы счастливы,
как семья по десятибалльной шкале?
58.5-60.4: - Восемь.
- Стойте.
60.6-64.8: Десятка - неописуемый восторг,
а единица - полное непонимание?
65.0-67.0: Постарайтесь ответить интуитивно.
67.2-68.9: Хорошо.
69.1-70.1: - Ты готова?
- Да.
70.4-71.6: - Восемь.
- Восемь.
74.8-77.1: Вы часто занимаетесь сексом?
77.3-79.5: Я не понимаю вопроса.
80.0-82.9: Да, непонятно.
По десятибалльной шкале?
83.1-87.0: Единица в этом случае -
очень-очень мало или совсем нет?
87.2-91.6: Ведь тогда вашу шкалу
нужно с нуля начинать.
95.1-97.8: Ну, хотя бы на этой неделе.
100

/tmp/ipython-input-5-2615003874.py:123: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = prev_sub.doc.similarity(curr_sub.doc) if prev_sub.doc and curr_sub.doc else 0
/tmp/ipython-input-5-2615003874.py:100: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = prev_scene[-1].doc.similarity(current_scene[0].doc)
/tmp/ipython-input-5-2615003874.py:83: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  sim = scene[i-1].doc.similarity(scene[i].doc)



Результат также сохранён в файл scenes_output.txt
